In [ ]:


# import import_ipynb
import numpy as np
from PreProcessing_v1 import import_tuh_abnormal
from DataTransform import data_transform, train_valid_test_split, get_parameters_for_model
import torch
from braindecode.datasets import BaseConcatDataset
from braindecode.preprocessing import preprocess, Preprocessor, scale as multiply
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, Deep4Net
from torch import optim
from Models import ShallowCNNLSTM
import seaborn as sns


# # Importing TUH Abnormal dataset

In [ ]:

TUH_PATH = 'C:/Users/Kitti/Documents/Thesis/TUH/Abnormal/SSample'
#TUH_PATH = 'TUH EEG Abnormal dataset/Abnormal/' # specify the path to the TUH Abnormal dataset
N_JOBS = 2  # specify the number of jobs for loading and windowing

# parameters to be defined for the preprocessing pipeline
TMIN = 1 * 60
TMAX = 5 * 60 
SFREQ = 100
CLIPPING = 800

In [ ]:


tuh_abnormal = import_tuh_abnormal(TUH_PATH, N_JOBS)

In [ ]:


def create_ch_mapping():
    short_ch_names = sorted([
        'A1', 'A2',
        'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
        'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'FZ', 'CZ', 'PZ'])
    ar_ch_names = sorted([
        'EEG A1-REF', 'EEG A2-REF',
        'EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF',
        'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF',
        'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF',
        'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'])
    le_ch_names = sorted([
        'EEG A1-LE', 'EEG A2-LE',
        'EEG FP1-LE', 'EEG FP2-LE', 'EEG F3-LE', 'EEG F4-LE', 'EEG C3-LE',
        'EEG C4-LE', 'EEG P3-LE', 'EEG P4-LE', 'EEG O1-LE', 'EEG O2-LE',
        'EEG F7-LE', 'EEG F8-LE', 'EEG T3-LE', 'EEG T4-LE', 'EEG T5-LE',
        'EEG T6-LE', 'EEG FZ-LE', 'EEG CZ-LE', 'EEG PZ-LE'])
    assert len(short_ch_names) == len(ar_ch_names) == len(le_ch_names)
    ar_ch_mapping = {ch_name: short_ch_name for ch_name, short_ch_name in zip(
        ar_ch_names, short_ch_names)}
    le_ch_mapping = {ch_name: short_ch_name for ch_name, short_ch_name in zip(
        le_ch_names, short_ch_names)}
    ch_mapping = {'ar': ar_ch_mapping, 'le': le_ch_mapping}    
    return short_ch_names, ch_mapping


def select_by_channels(ds, short_ch_names, ch_mapping):
    split_ids = []
    for i, d in enumerate(ds.datasets):
        ref = 'ar' if d.raw.ch_names[0].endswith('-REF') else 'le'
        # these are the channels we are looking for
        seta = set(ch_mapping[ref].keys())
        # these are the channels of the recoding
        setb = set(d.raw.ch_names)
        # if recording contains all channels we are looking for, include it
        if seta.issubset(setb):
            split_ids.append(i)
    return ds.split(split_ids)['0']


def custom_crop(raw, tmin=0.0, tmax=None, include_tmax=True):
    # crop recordings to tmin – tmax. can be incomplete if recording
    # has lower duration than tmax
    # by default mne fails if tmax is bigger than duration
    tmax = min((raw.n_times - 1) / raw.info['sfreq'], tmax)
    raw.crop(tmin=tmin, tmax=tmax, include_tmax=include_tmax)
    
    
def custom_rename_channels(raw, mapping):
    # rename channels which are dependent on referencing:
    # le: EEG 01-LE, ar: EEG 01-REF
    # mne fails if the mapping contains channels as keys that are not present
    # in the raw
    reference = raw.ch_names[0].split('-')[-1].lower()
    assert reference in ['le', 'ref'], 'unexpected referencing'
    reference = 'le' if reference == 'le' else 'ar'
    raw.rename_channels(mapping[reference])
    
short_ch_names, ch_mapping = create_ch_mapping()

preprocessors = [
        Preprocessor(custom_crop, tmin=TMIN, tmax=TMAX, include_tmax=True,
                     apply_on_array=False),
        Preprocessor('set_eeg_reference', ref_channels='average', ch_type='eeg'), # mne Raw class function
        Preprocessor(custom_rename_channels, mapping=ch_mapping, # rename channels to short channel names
                     apply_on_array=False), #
        Preprocessor('pick_channels', ch_names=short_ch_names, ordered=True), # mne Raw class function
        Preprocessor(multiply, factor=1e6, apply_on_array=True), # scaling signals to microvolt
        Preprocessor(np.clip, a_min=-CLIPPING, a_max=CLIPPING, apply_on_array=True), # clip outlier values to +/- 800 micro volts
        Preprocessor('resample', sfreq=SFREQ), # mne Raw class function
    ]

In [ ]:


tuh_preproc = preprocess(
        concat_ds=tuh_abnormal,
        preprocessors=preprocessors,
        n_jobs=N_JOBS,
        save_dir=None
    )


# # Data Transform

In [ ]:



# from braindecode.datasets.tuh import TUHAbnormal, TUH
# from braindecode.models import get_output_shape

In [ ]:


tuh_train, tuh_val, tuh_test = train_valid_test_split(tuh_preproc, train_size=0.9)



In [ ]:


# Extract number of chans and time steps from dataset
in_chans, input_size_samples = get_parameters_for_model(tuh_train)

In [ ]:


TRAIN_SIZE=0.9
INPUT_WIN_SAMPLES = 6000
BATCH_SIZE=64

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
# Set random seed to be able to roughly reproduce results
# Note that with cudnn benchmark set to True, GPU indeterminism
# may still make results substantially different between runs.
# To obtain more consistent results at the cost of increased computation time,
# you can set `cudnn_benchmark=False` in `set_random_seeds`
# or remove `torch.backends.cudnn.benchmark = True`
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 2


modelh = ShallowCNNLSTM(
    in_chans,
    n_classes,
    input_window_samples=INPUT_WIN_SAMPLES,
    final_conv_length=25,
)


# Send model to GPU
if cuda:
    modelh.to(device)

In [ ]:
model = ShallowFBCSPNet(
    in_chans,
    n_classes,
    input_window_samples=INPUT_WIN_SAMPLES,
    final_conv_length=25,
)
    
    

In [ ]:
train_loader, val_loader, test_loader = data_transform(tuh_train, tuh_val, tuh_test, model, train_size=TRAIN_SIZE, batch_size=BATCH_SIZE, input_window_samples=INPUT_WIN_SAMPLES, in_chans = in_chans)

In [ ]:


len(train_loader)


# # Training and testing

In [ ]:


from TrainingValidationTesting import train_model, test_model

In [ ]:


# hyperparameters for training the model
lr =  0.01
optimizer = optim.Adam(params=model.parameters(), lr=lr)
criterion = torch.nn.NLLLoss()
n_epochs = 1  
patience = 10

# Using multiple GPUs
# model = torch.nn.DataParallel(model)

In [ ]:



trained_model, train_losses, train_accs, valid_losses = train_model(model, device, criterion, optimizer, train_loader, val_loader, n_epochs, patience)

In [ ]:


trained_model.to(device)



In [ ]:


x_test, y_test, y_pred, accuracy = test_model(device, trained_model, test_loader)

In [ ]:


# Save the trained model

PATH = './cnnshallow.pth'
# torch.save(model.state_dict(), PATH)

#how to reload the saved model:

model = ShallowFBCSPNet(
    in_chans,
    n_classes,
    input_window_samples=INPUT_WIN_SAMPLES,
    final_conv_length=1,
)

model.load_state_dict(torch.load(PATH))

In [ ]:
x_test, y_test, y_pred, accuracy = test_model(device, model, test_loader)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, RocCurveDisplay

In [ ]:

x_test = x_test.flatten().cpu()
y_test = y_test.flatten().cpu()
y_pred = y_pred.flatten().cpu()

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots()
ax.plot(train_losses, marker="o", color="orangered", label="Training loss")
ax.set_xlabel("Number of epochs")
ax.set_ylabel("Loss", color="orangered")

# twin object for two different y-axis on the sample plot
ax2=ax.twinx()
ax2.plot(train_accs, marker="o", color="cornflowerblue", label="Training accuracy")
ax2.set_ylabel("Accuracy", color="cornflowerblue")

plt.title("Training loss and accuracy per epoch", fontsize=14)
plt.show()

# save the plot as a file
'''
fig.savefig('two_different_y_axis_for_single_python_plot_with_twinx.jpg',
            format='jpeg',
            dpi=100,
            bbox_inches='tight')
'''



# In[87]: Losses plot

plt.plot(train_losses, marker=".", color="cornflowerblue", label='Training loss')
plt.plot(valid_losses, marker=".", color="mediumseagreen", label='Validation loss')
plt.title('Training and Validation Losses', fontsize=14)
plt.xlabel('Number of epochs')
plt.legend(frameon=False)
plt.show()



In [ ]:
''' 
The ROC Curve plots two parameters: the True Positive Rate (TPR), a synonim to recall,
and the False Positive Rate (FPR). 
The plot also shows the ROC AUC score.
By default, the class considered as the positive (It means Abnormal in our case).

'''

RocCurveDisplay.from_predictions(y_test, y_pred, pos_label=None)
plt.show()


# In[90]: Confusion matrix and Classification Report

conf_matrix = confusion_matrix(y_test, y_pred)
cl_report = classification_report(y_test, y_pred)

In [ ]:

print(cl_report)

In [ ]:

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                conf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     conf_matrix.flatten()/np.sum(conf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(conf_matrix, annot=labels, fmt='', cmap='YlGn')




